<a href="https://colab.research.google.com/github/ernesto-butto/ag2-workshop-simulation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


- Phase 1: Scenario Generation Agents
- Phase 2: Running the Scenario with a Game Master
- Phase 3: Live Playtesting
- Phase 4: AI Feedback Agent (Optional)

Instalar AG2
https://docs.ag2.ai/docs/home/quick-start

In [22]:
!pip install ag2

Setup

In [23]:
# 1. Import our agent class
from autogen import ConversableAgent
from pprint import pprint

# Surpress warnings
import warnings
warnings.filterwarnings("ignore")


In [45]:
llm_config = {"config_list": [{
    "model": "gpt-4o-mini",  # This should match your Azure deployment name
    "api_key": "0d20814cba1242d39825274ccd99d2e5",
    "base_url": "https://ai-ernestobuttoai916354951815.services.ai.azure.com/",
    "api_type": "azure",
    "api_version": "2024-08-01-preview",
    "temperature": 0.8
}]}

# Phase 1: Scenario Generation Agents
🔹 **Attendees Build:**

- **(A) Scenario Creation Agent** → Generates a simulation scenario.
- **(B) Mocked User Agent** → Simulates a user providing input for scenario generation.

🔹 **Simulation I (Test Run)**

- A & B **interact automatically** → Produces a detailed scenario.
- Attendees analyze the **output** to understand if it’s coherent and engaging.

In [46]:
fin_de_conversacion_keyword = "FIN_ESCENARIO"

In [47]:
def createScenarioCreationAgent():

  return ConversableAgent(
      name="scenario_creation_agent",
      llm_config=llm_config,
      code_execution_config=False,
      human_input_mode="NEVER",
      system_message=f"""
      Eres un creador de escenarios para simulaciones conversacionales 1 a 1.
      Tu objetivo es ayudar a los estudiantes a crear escenarios de juegos de roles para practicar habilidades basado en sus hobbies e intereses.
      Explicale tu objetivo al estudiante que te salude, y hazle preguntas sobre:

      1. Que habilidades quiere practicar
      2. Algún interes o hobby para crear un escenario que pueda relacionar

      Algunos puntos a tomar en cuenta:
      - El escenario debe de ser cortos, y con un objetivo claro, que para lograrlo debe conversar con otra persona.
      - Una vez tengas la información necesaria, genera un escenario y preguntale al estudiante si está deacuerdo
      - Itera con el usuario, cuando se llegue a un escenario que le guste, escribe todo el escenario, terminado la conversación con el término: {fin_de_conversacion_keyword}
      - Nunca empiezes el escenario, siempre termina la conversación con {fin_de_conversacion_keyword} cuando el escenario esté listo.

      """,
      is_termination_msg=lambda msg: fin_de_conversacion_keyword in msg["content"],
  )


In [48]:
def mockedUserAgent(skill, hobby):
  return ConversableAgent(
        "mocked_student",
        llm_config=llm_config,
        human_input_mode="NEVER",  # always ask for human input
        system_message=f"You are a student learning {skill}, and you like {hobby}",
        code_execution_config=False,
        is_termination_msg=lambda msg: fin_de_conversacion_keyword in msg["content"],
    )

In [49]:
creador = createScenarioCreationAgent()
estudiante = mockedUserAgent("Negotiation", "90's pop music")
chat_result = estudiante.initiate_chat(
    creador,
    message="Hola!",
    max_turns=12,
    summary_method="reflection_with_llm",
    summary_prompt="Escribe el escenario en Español"
)


mocked_student (to scenario_creation_agent):

Hola!

--------------------------------------------------------------------------------
scenario_creation_agent (to mocked_student):

¡Hola! Soy un creador de escenarios para simulaciones conversacionales. Mi objetivo es ayudarte a practicar habilidades a través de juegos de roles, basándome en tus intereses y hobbies. 

Para comenzar, ¿qué habilidades te gustaría practicar? Y también, ¿tienes algún interés o hobby que te gustaría que vinculáramos al escenario?

--------------------------------------------------------------------------------
mocked_student (to scenario_creation_agent):

¡Hola! Me gustaría practicar habilidades de negociación. Como me gusta la música pop de los 90, podríamos crear un escenario en el que esté negociando con un amigo sobre el reparto de canciones para una fiesta temática de los 90. ¿Te parece bien?

--------------------------------------------------------------------------------
scenario_creation_agent (to moc

In [50]:

print("HISTORY")
print(chat_result.chat_history)
print()

print("SUMMARY")
pprint(chat_result.summary)
print()

print("COST")
pprint(chat_result.cost)
print()

HISTORY
[{'content': 'Hola!', 'role': 'assistant', 'name': 'mocked_student'}, {'content': '¡Hola! Soy un creador de escenarios para simulaciones conversacionales. Mi objetivo es ayudarte a practicar habilidades a través de juegos de roles, basándome en tus intereses y hobbies. \n\nPara comenzar, ¿qué habilidades te gustaría practicar? Y también, ¿tienes algún interés o hobby que te gustaría que vinculáramos al escenario?', 'role': 'user', 'name': 'scenario_creation_agent'}, {'content': '¡Hola! Me gustaría practicar habilidades de negociación. Como me gusta la música pop de los 90, podríamos crear un escenario en el que esté negociando con un amigo sobre el reparto de canciones para una fiesta temática de los 90. ¿Te parece bien?', 'role': 'assistant', 'name': 'mocked_student'}, {'content': '¡Eso suena genial! La música pop de los 90 es un tema divertido para un escenario de negociación. Entonces, para aclarar un poco más, ¿cuál sería el objetivo de la negociación? ¿Quieres asegurarte d